Мультитредінговий конвертер валют
Розробіть програму, яка використовує мультитредінг для одночасного
конвертування валют з одного формату в інший. Програма має зчитувати
валютні курси з файлу (наприклад, CSV) та конвертувати суми у різних потоках.
Реалізуйте синхронізацію між потоками для уникнення конфліктів при доступі до спільних даних.

In [2]:
import csv
import threading
import time


class CurrencyConverter:
    def __init__(self, source_currency, exchange_rates):
        self.source_currency = source_currency
        self.exchange_rates = exchange_rates
        self.lock = threading.Lock()

    def convert(self, target_currency, amount):
        if target_currency == self.source_currency:
            return amount
        with self.lock:
            return amount * self.exchange_rates[target_currency]


def read_exchange_rates(filename):
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        exchange_rates = {}
        for row in reader:
            currency = row.pop("Currency")
            exchange_rates[currency] = {k: float(v) for k, v in row.items()}
    return exchange_rates


def main():
    filename = "exchange_rates.csv"
    exchange_rates = read_exchange_rates(filename)
    currency_converters = {}
    for source_currency, currency_data in exchange_rates.items():
        currency_converters[source_currency] = CurrencyConverter(source_currency, currency_data)

    print("Доступні коди валют:", list(currency_converters.keys()))

    conversion_threads = []
    for source_currency, target_currency, amount in [
        ("USD", "EUR", 100),
        ("EUR", "PLN", 50),
        ("PLN", "UAH", 20),
        ("UAH", "GBP", 100),
        ("GBP", "USD", 75),
    ]:
        currency_converter = currency_converters.get(source_currency)
        if currency_converter is None:
            print(f"Конвертер валюти не знайдено для '{source_currency}'")
        else:
            thread = threading.Thread(target=convert_in_thread,
                                      args=(currency_converter, source_currency, target_currency, amount))
            conversion_threads.append(thread)
            thread.start()

    for thread in conversion_threads:
        thread.join()


def convert_in_thread(currency_converter, source_currency, target_currency, amount):
    print(f"Початок конвертації {source_currency} в {target_currency}")
    time.sleep(1)  # Затримка в одну секунду для демонстрації паралельності
    result = currency_converter.convert(target_currency, amount)
    print(f"Завершення конвертації {source_currency} в {target_currency}")
    print(f"Конвертація з {source_currency} в {target_currency}: {result}")


if __name__ == "__main__":
    main()


Доступні коди валют: ['USD', 'EUR', 'PLN', 'UAH', 'GBP']
Початок конвертації USD в EUR
Початок конвертації EUR в PLN
Початок конвертації PLN в UAH
Початок конвертації UAH в GBP
Початок конвертації GBP в USD
Завершення конвертації PLN в UAH
Конвертація з PLN в UAH: 3.2
Завершення конвертації GBP в USD
Конвертація з GBP в USD: 91.5
Завершення конвертації USD в EUR
Конвертація з USD в EUR: 88.0
Завершення конвертації EUR в PLN
Конвертація з EUR в PLN: 12.0
Завершення конвертації UAH в GBP
Конвертація з UAH в GBP: 2178.0
